In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, \
TensorDataset, random_split
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import io

In [27]:
from google.colab import files
uploaded = files.upload()

Saving iris_data.csv to iris_data.csv


In [29]:
df = pd.read_csv(io.BytesIO(uploaded['iris_data.csv']))

In [30]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,SetosaScore,VersicolorScore,VirginicaScore
0,1,5.5,2.5,4.0,1.3,0.085468,0.906529,-0.022924
1,2,6.9,3.1,5.4,2.1,0.107538,-0.093120,1.150347
2,3,5.8,2.8,5.1,2.4,0.166387,0.089225,0.890712
3,4,6.3,2.8,5.1,1.5,0.131216,-0.050547,0.821305
4,5,4.9,2.5,4.5,1.7,-0.192957,-0.081149,0.970756
...,...,...,...,...,...,...,...,...
145,146,4.4,3.2,1.3,0.2,0.953548,-0.166344,-0.058804
146,147,5.0,3.3,1.4,0.2,1.105043,0.134535,-0.123027
147,148,5.8,2.7,4.1,1.0,-0.026048,0.930779,0.029497
148,149,5.1,3.3,1.7,0.5,1.112032,-0.148318,0.192130


In [39]:
#convert data to np and x and Y
data = df.to_numpy()

data.shape
data = data[:, 1: ]

X = data[:, :-3]
y = data[:, 4:]

In [60]:
# Split the data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Standardize the training data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Apply the same transformation to the validation and test data
X_test = scaler.transform(X_test)

In [69]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [72]:
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [73]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [92]:
#neural net
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 512)
        self.fc3 = nn.Linear(512, 3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [93]:
# Instantiate the model
model = IrisNet()

# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),
                       lr=0.001,
                       weight_decay=0.01)


In [96]:
# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # This loop iterates over the
        # mini-batches of data provided
        # by train_loader. Each mini-batch
        # contains a small subset of the
        # training data.
        optimizer.zero_grad()

        # The forward pass is performed here.
        # The model processes the inputs and
        # produces outputs (predictions).
        outputs = model(inputs)

        # The loss function (defined by criterion)
        # calculates the difference between the
        # model's predictions (outputs) and the
        # true labels (labels). The result is
        # the loss, which indicates how well
        # the model's predictions match
        # the actual values.
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()

In [97]:
#training loss
print(loss)

tensor(0.2654, grad_fn=<MeanBackward0>)


In [98]:
# Evaluate the model on the test set
model.eval()
test_loss = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

print(f'Test Loss: {test_loss / len(test_loader)}')

Test Loss: 0.19177060574293137


In [105]:
q1 = torch.tensor([7.0,4.0,7.0,3.0])
with torch.no_grad():
  out = model(q1)
  print(out)

tensor([-0.1714, -0.3308,  1.5229])
